# Networks and their Structure Assignment

## Network Science Topic 2

Note that the networks in this exercise are undirected.

This exercise concerns the **Kleinberg model** which we claimed was an example of a *searchable* network.

A **Kleinberg graph** is defined by four parameters $n$, $m$, $p$ and $\alpha$.  It is constructed as follows:
* The graph has $n$ nodes. We think of these as being arranged on a circle.
* Each node is joined to the nearest $m$ nodes on either side (so it has $2m$ neighbours in total).
* Then each node $v$ is considered, and each of the original edges going to a clockwise neighbour is considered in turn. With rewiring probability $p$ the edge is deleted and replaced with an edge from $v$ to a vertex $w$ chosen at random: but vertices are not equally likely to be chosen as $w$. The vertex $w$ is chosen with probability proportional to $d_w^{-\alpha}$ where $d_w$ is the shorter distance around the circle from v to w. (Note that if the edge from $v$ to $w$ already exists then the original edge is not deleted and no action is taken).

Write code to construct an instance of the Kleinberg model.  Here is how you might find the probability distribution needed to choose the random vertex $w$ in the third step above.
* Once you have decided to rewire an edge from $v$, for every other vertex $w$, find $d_w^{-\alpha}$. Put these values in a list.
* Find the sum $S$ of the values in the list. So the probability of rewiring from $v$ to $w$ is $d_w^{-\alpha}/S$.
* Find a random real number $R$ between 0 and $S$, and initiate a variable *total* with value 0.
* Go through the list, add each number to *total*. Stop when *total* is more than $R$. The vertex that corresponds to the point reached in the list is $w$.

In [ ]:
import random
import itertools
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%run NatSfunctions.ipynb

In [ ]:
# delete the current edge and replace with edge to another vertex
def rewire(v, neighbour, n, a, graph):
    
    distances = []
    
    # find the shorter distance from v to every other vertex raised to power -a
    for vertex in graph:

        # compute distances in both directions; take minimum and raise to -a
        distance1 = abs(vertex - v)
        distance2 = (distance1 * -1) % n

        a_distance = min(distance1, distance2)
        
        if a_distance != 0:
            a_distance = a_distance ** -a

        distances.append(a_distance)
    
    # compute probability
    S = sum(distances)
    random_number = random.uniform(0, S)
    total = 0
    
    # go through list and add each number to total
    for i in range(0, len(distances)):
        
        total += distances[i]
        if total > random_number:
            w = i  # vertex corresponding to this point is the one rewired to
            break
    
    # if edge from v to w already exists do nothing
    # otherwise remove the old edge and add the rewired one
    if w not in graph[v]:
        graph[v].append(w)
        graph[v].remove(neighbour)

    return graph

In [ ]:
def create_Kleinberg_graph(n, m, p, a):

    graph = {}

    # connect initial vertices to m nearest neighbours on either side
    for vertex in range(0, n):

        # get the neighbours ahead and behind current vertex
        clockwise_neighbours = [neighbour % n for neighbour in range(vertex + 1, vertex + m + 1)]
        anticlockwise_neighbours = [neighbour % n for neighbour in range(vertex - 1, vertex - m - 1, -1)]

        # combine and assign to vertex
        neighbours = clockwise_neighbours + anticlockwise_neighbours
        graph[vertex] = neighbours


    # consider each node
    for vertex in graph:
        
        # consider each edge going to a clockwise neighbour
        clockwise_neighbours = graph[vertex][:m]
        
        for neighbour in clockwise_neighbours:
            
            # rewire with rewiring probability p
            random_number = random.random()
            if random_number < p:
                rewired_graph = rewire(vertex, neighbour, n, a, graph)
    
    return graph

Write a function to find the *search time* of a graph. This is the number of steps needed to get from a vertex $v$ to a vertex $w$ when, at each step, you traverse the edge that will bring you to the position on the circle closest to $w$. In fact, the search time is the average over all pairs of vertices, but if this is too slow to compute it should be possible to approximate by look at a random sample of pairs.

In [ ]:
# returns number of steps needed to get from v to w
# when traversing at each step the edge whose endpoint is closest to w on the circle
def get_steps(v, w, graph):
    
    # initialise steps and neighbours
    steps = 0
    
    while True:

        neighbours = graph[v]

        # if w is a neighbour then steps is 1
        if w in neighbours:
            steps += 1
            break

        # otherwise find closest neighbour to w and keep going
        v = min(neighbours, key=lambda x:abs(x - w))
        steps += 1

    return steps

In [ ]:
# average number of steps between all pairs of vertices
def get_search_time(graph):

    steps_list = []
    vertex_combinations = list(itertools.combinations(graph, 2))

    for v, w in vertex_combinations:
        steps = get_steps(v, w, graph)
        steps_list.append(steps)
    
    average_steps = sum(steps_list)/len(steps_list)
    return average_steps

1. [15 marks] Plot search time versus $\alpha$ for Kleinberg graphs with $m=8$, $p=0.05$ for the following values of $n$: 1000, 2000, 3000, 4000.

In [24]:
n_values = [1000, 2000, 3000, 4000]
m = 8
p = 0.05
xdata = [[], [], [], []]
ydata = [[], [], [], []]

for index in range(0, len(n_values)):

    n = n_values[index]
    
    for a in np.arange(0, 5, 0.1):  # TODO: what range and steps for a?

        K_graph = create_Kleinberg_graph(n, m, p, a)
        search_time = get_search_time(K_graph)
        
        xdata[index].append(a)
        ydata[index].append(search_time)
        print(a, search_time)

    print(xdata, ydata)


0.0 13.92391991991992
0.1 14.62867067067067
0.2 15.106918918918918
0.30000000000000004 15.55837037037037
0.4 14.682082082082083
0.5 15.678738738738739
0.6000000000000001 15.34704904904905
0.7000000000000001 16.356262262262263
0.8 17.037843843843845
0.9 20.33178778778779
1.0 20.946766766766768
1.1 20.54843243243243
1.2000000000000002 25.718774774774776
1.3 27.686728728728728
1.4000000000000001 33.34303103103103
1.5 33.96818018018018
1.6 37.87835435435436
1.7000000000000002 37.32124924924925
1.8 37.00778778778779
1.9000000000000001 39.729073073073074
2.0 38.362554554554556
2.1 40.76471071071071
2.2 38.57543543543544
2.3000000000000003 41.32226826826827
2.4000000000000004 41.43145945945946
2.5 41.47639239239239
2.6 41.58703103103103
2.7 41.50992392392392
2.8000000000000003 41.456606606606606
2.9000000000000004 41.65445645645646
3.0 41.64696296296297
3.1 41.56898098098098
3.2 41.648746746746745
3.3000000000000003 41.64397197197197
3.4000000000000004 41.62305105105105
3.5 41.62983983983984


KeyboardInterrupt: 

2. [5 marks] Plot the relationship between the number of nodes and search time (for fixed $m$, $p$ and $\alpha$).

You will need to submit all code and plots for this question.  You can do this most simply by extending this notebook, but you can also, for example, submit py and image files.  You will be given detailed instructions on the submission for this module before the deadline next term.